<a href="https://colab.research.google.com/github/mahalakshmimalla/Assignment-1/blob/main/Glucouma-detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ValueError: mount failed

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage.util import random_noise
import os
import random

def add_noise_to_image(image, dB=25):  # Fixed noise level to 25 dB
    # Choose a random noise type from the available options
    noise_types = ['uniform', 'pepper', 'gaussian']
    noise_type = random.choice(noise_types)

    if noise_type == 'uniform':
        def dB_to_noise_range(dB):
            max_dB = 30  # Set the maximum dB value
            noise_magnitude = dB / max_dB
            return (-noise_magnitude * 255, noise_magnitude * 255)

        def add_uniform_noise(image, noise_range):
            noisy_image = np.copy(image)  # Work with color image directly
            min_noise, max_noise = noise_range
            noise = np.random.uniform(min_noise, max_noise, image.shape)
            noisy_image = np.clip(image + noise, 0, 255)
            return noisy_image.astype(np.uint8)

        noise_range = dB_to_noise_range(dB)
        noisy_image = add_uniform_noise(image, noise_range)

    elif noise_type == 'pepper':
        def dB_to_noise_level(dB):
            max_dB = 30
            noise_level = dB / max_dB
            return np.clip(noise_level, 0, 1)

        def add_pepper_noise(image, noise_level):
            noisy_image = np.copy(image)
            total_pixels = image.size // 3  # Each pixel has 3 channels (R, G, B)
            num_pepper = int(total_pixels * noise_level)
            pepper_indices = np.random.choice(total_pixels, num_pepper, replace=False)
            noisy_image.reshape(-1, 3)[pepper_indices, :] = 0  # Set pepper noise to black
            return noisy_image

        noise_level = dB_to_noise_level(dB)
        noisy_image = add_pepper_noise(image, noise_level)

    elif noise_type == 'gaussian':
        def db_to_std(db):
            max_dB = 30
            min_std = 0.0001
            max_std = 0.1
            std = min_std + (db / max_dB) * (max_std - min_std)
            return std

        std_dev = db_to_std(dB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_np = image_rgb / 255.0
        noisy_image_np = random_noise(image_np, mode='gaussian', var=std_dev**2)
        noisy_image_np = np.clip(noisy_image_np * 255, 0, 255).astype(np.uint8)
        noisy_image = noisy_image_np  # Return as NumPy array instead of PIL Image

    return noisy_image, noise_type

def process_images_from_dataset(dataset_path):
    images = os.listdir(dataset_path)

    for image_file in images:
        image_path = os.path.join(dataset_path, image_file)
        if not os.path.isfile(image_path):
            print(f"File {image_path} is not a valid image.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Could not read image {image_path}.")
            continue

        noisy_image, noise_type = add_noise_to_image(image, dB=5)  # Always use 25 dB noise

        # Display the original and noisy images
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title('Original Image')
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f'Noisy Image ({noise_type})')
        plt.imshow(cv2.cvtColor(noisy_image, cv2.COLOR_BGR2RGB))  # Now works with NumPy array
        plt.axis('off')

        # Assuming image file names are related to iris disease type or patient ID
        plt.suptitle(f"Iris Disease Identifier: {os.path.splitext(image_file)[0]}")
        plt.show()

if __name__ == "__main__":
    dataset_path = '/content/gdrive/MyDrive/full_df.csv/full_df.csv'  # Path to the dataset folder containing images
    process_images_from_dataset(dataset_path)


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image


In [ ]:
def preprocess_image(img_path, img_height=224, img_width=224):
    # Load the image
    img = Image.open(img_path)
    # Resize the image to the target size
    img = img.resize((img_height, img_width))
    # Convert the image to an array and scale it
    img_array = image.img_to_array(img) / 255.0
    # Expand dimensions to match the input shape of the model (1, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


In [ ]:
def predict_glaucoma(model, img_path):
    # Preprocess the image
    img_array = preprocess_image(img_path)
    # Make a prediction
    prediction = model.predict(img_array)
    # Interpret the result
    if prediction[0][0] >= 0.5:
        result = "Glaucoma detected"
    else:
        result = "No Glaucoma detected"
    print(f"Prediction: {result}")


NameError: name 'model' is not defined

In [ ]:
import tensorflow as tf  # Import TensorFlow
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# Function to preprocess a single image
def preprocess_image(img_path, img_height=224, img_width=224):
    img = Image.open(img_path)
    img = img.resize((img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict glaucoma
def predict_glaucoma(model, img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    if prediction[0][0] >= 0.5:
        result = "Glaucoma detected"
    else:
        result = "No Glaucoma detected"
    print(f"Prediction: {result}")

# Load the trained model
model = tf.keras.models.load_model('glaucoma_detection_model.h5')

# Path to the image you want to test
img_path = '/content/gdrive/MyDrive/archive/dataset/glaucoma/1209_right.jpg'

# Run the prediction
predict_glaucoma(model, img_path)
# Save the model after training
model.save('glaucoma_detection_model.h5')



FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'glaucoma_detection_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:

import os
print(os.listdir())  # Should list 'glaucoma_detection_model.h5' if it exists


['.config', 'gdrive', 'sample_data']
